In [2]:
import psycopg2
import time
import boto3
import copy
import gc
import pandas as pd

#### Code to Count Records of CSV

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

cnt = 0
for i, row in enumerate(obj["Body"].iter_lines()):
    if i==0:
        continue
    cnt += 1
    if (i%10_000_000)==0:
        print(f"{i} Records")

In [4]:
total_records = 979949280

total_records

979949280

In [3]:
db_connection = psycopg2.connect(
            user='USER_STRING_HERE', password='PW_STRING_HERE', #Commented out for Github upload
            host='mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com', port=5432,
            database="mads"
        )

db_connection.autocommit = True

cursor = db_connection.cursor()

In [29]:
delete_query = "DROP TABLE greeks5"
cursor.execute(delete_query)

In [30]:
query = '''CREATE TABLE greeks5 (id SERIAL PRIMARY KEY,  
                                exec_date date,
                                increment SMALLINT,
                                opt_value float,
                                ticker VARCHAR(8),
                                greek VARCHAR(16),
                                option_type VARCHAR(8))'''


cursor.execute(query)

%%time

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

insert_query = "INSERT INTO greeks4 (exec_date, increment, opt_value, ticker, greek, option_type) values ('{}',{},{},'{}','{}','{}');"

long_query = ""

for i, row in enumerate(obj["Body"].iter_lines()):
    if i == 0:
        continue
    if i<=3000000: ## First Run failed at # Mn records. Condition to skip records till last fail count.
        if (i%1_000_000) ==0:
            print(f"{i} Records Passed.")
        continue
    try:
        row_data = row.decode().split(",")
        dt, incr, opt_value, ticker, greek, opt_type = row_data
        dt = dt.split()[0]
        incr = int(incr) if incr else "NULL"
        opt_value = float(opt_value) if opt_value else "NULL"
    except Exception as e:
        print(e)
        print(f"{i} - {row_data} Failed Parsing.")
        continue
        
    query = insert_query.format(dt,incr,opt_value,ticker,greek,opt_type)
    long_query += copy.copy(query + "\n")
    #print(query)
    if (i%100_000)==0: ## Batching 100k records and uploading all at once.
        cursor.execute(long_query)
        long_query = ""
        gc.collect();
        if (i%1_000_000) == 0:
            print(f"{i} Records Inserted")
        

if long_query:
    cursor.execute(long_query)

In [31]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks5"

cursor.execute(select_query)

cursor.fetchall()

[(0,)]

In [56]:
#query = "SELECT aws_s3.table_import_from_s3('greeks5','','(format csv)', '(mads-capstone-2023,greeks2.csv,us-east-1)');"

#cursor.execute(select_query)

In [32]:
%%time

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []

for i, row in enumerate(obj["Body"].iter_lines()):
    if i == 0:
        continue
    #if i<=3000000: ## First Run failed at # Mn records. Condition to skip records till last fail count.
    #    if (i%1_000_000) ==0:
    #        print(f"{i} Records Passed.")
    #    continue
    try:
        row_data = row.decode().split(",")
        dt, incr, opt_value, ticker, greek, opt_type = row_data
        dt = dt.split()[0]
        incr = int(incr) if incr else None
        opt_value = float(opt_value) if opt_value else None
    except Exception as e:
        print(e)
        print(f"{i} - {row_data} Failed Parsing.")
        continue

    records.append([dt,incr,opt_value,ticker,greek,opt_type])

    if (i%100_000)==0: ## Batching 100k records and uploading all at once.
        df = pd.DataFrame(records)#.dropna()
        df.to_csv("sample.csv", header=False, index=False)
        with open("sample.csv", "rb") as fp:
            cursor.copy_from(fp, "greeks5", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

        records = []
        gc.collect();

        if (i%1_000_000) == 0:
            print(f"{i} Records Inserted")

if records:
    df = pd.DataFrame(records).dropna()
    df.to_csv("sample.csv", header=False, index=False)
    with open("sample.csv", "rb") as fp:
        cursor.copy_from(fp, "greeks5", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

1000000 Records Inserted
2000000 Records Inserted
3000000 Records Inserted
4000000 Records Inserted
5000000 Records Inserted
6000000 Records Inserted
7000000 Records Inserted
8000000 Records Inserted
9000000 Records Inserted
10000000 Records Inserted
11000000 Records Inserted
12000000 Records Inserted
13000000 Records Inserted
14000000 Records Inserted
15000000 Records Inserted
16000000 Records Inserted
17000000 Records Inserted
18000000 Records Inserted
19000000 Records Inserted
20000000 Records Inserted
21000000 Records Inserted
22000000 Records Inserted
23000000 Records Inserted
24000000 Records Inserted
25000000 Records Inserted
26000000 Records Inserted
27000000 Records Inserted
28000000 Records Inserted
29000000 Records Inserted
30000000 Records Inserted
31000000 Records Inserted
32000000 Records Inserted
33000000 Records Inserted
34000000 Records Inserted
35000000 Records Inserted
36000000 Records Inserted
37000000 Records Inserted
38000000 Records Inserted
39000000 Records Inse

KeyboardInterrupt: 

In [33]:
if records:
    df = pd.DataFrame(records).dropna()
    df.to_csv("sample.csv", header=False, index=False)
    with open("sample.csv", "rb") as fp:
        cursor.copy_from(fp, "greeks5", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

KeyboardInterrupt: 

In [ ]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks5"

cursor.execute(select_query)

cursor.fetchall()

In [26]:
#bucket = 'mads-capstone-2023' 
#data_key = 'greeks2.csv' 

#s3_client = boto3.client('s3')
#obj = s3_client.get_object(Bucket=bucket, Key=data_key)

#cursor.copy_from(obj["Body"], "greeks5", sep=",", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

In [4]:
db_connection.close()